In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import cv2
cwd = os.getcwd()
os.chdir("C:/Users/toranj/Desktop/Persian Mnist")

In [11]:
Seperate("one","New Folder")

In [10]:
def Seperate(loadfile,savefolder):   
    img = cv2.imread(loadfile + ".jpg",0)
    img = 255 - img
    ydone = False
    start_ypoints_crop = []
    end_ypoints_crop = []
    for row in range(len(img)):
        if any([ind < 200 for ind in iter(img[row])]) and not ydone:
            start_ypoints_crop.append(row)
            ydone = True
        if all([ind > 200 for ind in iter(img[row])]) and ydone:
            end_ypoints_crop.append(row)
            ydone = False

    CroppedImages= []
    for top,bot in zip(start_ypoints_crop,end_ypoints_crop):
        CroppedImages.append(img[top:bot,...])

    xdone = False
    start_xpoints_crop = []
    end_xpoints_crop = []
    for k in range(len(CroppedImages)):
        start_xpoints_crop.append([])
        end_xpoints_crop.append([])
    for i in range(len(CroppedImages)):
        for col in range(len(CroppedImages[0][0])):
            if any([ind < 200 for ind in iter(CroppedImages[i][...,col])]) and not xdone:
                start_xpoints_crop [i].append(col)
                xdone = True
            if all([ind > 200 for ind in iter(CroppedImages[i][...,col])]) and xdone:
                end_xpoints_crop [i].append(col)
                xdone = False

    Numbers=[]
    for i in range(len(CroppedImages)):
        for left,right in zip(start_xpoints_crop[i],end_xpoints_crop[i]):
            Numbers.append(CroppedImages[i][...,left-2:right+2])

    for i in range(len(Numbers)):
        cv2.copyMakeBorder(Numbers[i],4,4,10,10,cv2.BORDER_CONSTANT,value=255)
        cv2.imwrite(f'Images/{savefolder}/{i}.png',Numbers[i])

In [5]:
def Show(image):
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [3]:
Seperate("plate","Test Char")

TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'

In [6]:
img = cv2.imread("plate.jpg",0)
print(img)

None


In [3]:
folders = ['Zeros','Ones','Twos','Threes','Fours','Fives','Sixes','Sevens','Eights','Nines']
final_images = []
for n in range(len(folders)):   
    for file in glob.glob(f"C:/Users/toranj/Desktop/Persian Mnist/Images/{folders[n]}/*.jpg"):
        image = cv2.imread(file,0)
        image = cv2.copyMakeBorder(image,2,2,10,10,cv2.BORDER_CONSTANT,value=255)
        image = cv2.resize(image,(28,28))                
        image = image / np.max(image)
        final_images.append(image)

In [4]:
from keras.utils import to_categorical
folders = ['Zeros','Ones','Twos','Threes','Fours','Fives','Sixes','Sevens','Eights','Nines']
labels = []
final_labels = []
for n in range(len(folders)):   
    for file in glob.glob(f"C:/Users/toranj/Desktop/Persian Mnist/Images/{folders[n]}/*.jpg"):
        labels.append(n)
final_labels = to_categorical(labels,num_classes = 10)


E:\Software\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
final_labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [5]:
from sklearn.model_selection import train_test_split
img_train, img_test, lbl_train, lbl_test = train_test_split(final_images, final_labels,
                                                            test_size=0.2, random_state=42, stratify=final_labels)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

CNT_Model = Sequential()


CNT_Model.add(Conv2D(20,5, activation = 'relu', padding = 'same',kernel_regularizer = None, input_shape =(28,28,1)))
CNT_Model.add(MaxPooling2D(pool_size = (2,2)))
CNT_Model.add(Conv2D(20,5, activation = 'relu', padding = 'same', kernel_regularizer = None))
CNT_Model.add(MaxPooling2D(pool_size = (2,2)))
#CNT_Model.add(Dropout(0.5))
CNT_Model.add(Flatten())
CNT_Model.add(Dropout(0.5))
#CNT_Model.add(BatchNormalization())
CNT_Model.add(Dense(64,activation = 'relu', kernel_regularizer = None))
#CNT_Model.add(BatchNormalization())
CNT_Model.add(Dense(10,activation='sigmoid'))

Optimizer = keras.optimizers.SGD(lr=0.01, momentum = 0.5, decay = 1e-5, nesterov = True)

CNT_Model.compile(optimizer=Optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
CNT_Model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 20)        10020     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 20)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 980)               0         
_________________________________________________________________
dropout_1 (Dropout)  

In [7]:
img_train = np.reshape(img_train,(len(img_train),28,28,1))

from keras.callbacks import ModelCheckpoint
ann_hist = CNT_Model.fit(np.array(img_train), np.array(lbl_train), validation_split = 0.1, batch_size = 10, epochs = 2)



Instructions for updating:
Use tf.cast instead.
Train on 9828 samples, validate on 1093 samples
Epoch 1/2
9828/9828 [==============================] - 8s 771us/step - loss: 1.1698 - acc: 0.5786 - val_loss: 0.1869 - val_acc: 0.9149
Epoch 2/2
9828/9828 [==============================] - 4s 369us/step - loss: 0.1357 - acc: 0.9571 - val_loss: 0.0230 - val_acc: 0.9954


In [8]:
img_test = np.reshape(img_test,(len(img_test),28,28,1))

test_loss, test_acc = CNT_Model.evaluate(np.array(img_test), np.array(lbl_test), batch_size = 3)
CNT_Model.save('Model1.h5')
labels_predicted = CNT_Model.predict(np.array(img_test))
lbl= np.argmax(labels_predicted, axis=1)
print('Test loss is',test_loss)
print('Test accuracy is',test_acc)

2731/2731 [==============================] - 1s 457us/step
Test loss is 0.015586355828267501
Test accuracy is 0.99853533512941


E:\Software\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
lbl[1]


0